In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
#기존 전처리파일 고유값1개인거 제거, 상관계수 1, 결측치 범주형 기타로대체, 데이터타입 변환 등등


df_trian = pd.read_parquet(r"C:\Users\rtc32\OneDrive\바탕 화면\final_project\project\3.승인매출정보_전처리완.parquet")
df_test = pd.read_parquet(r"C:\Users\rtc32\OneDrive\바탕 화면\final_project\project\3.승인매출정보_전처리완_test.parquet")

In [73]:
df_test.shape

(600000, 25)

In [4]:
#Segment 추가를 위해 회원정보 컬럼과 결합을 위해 불러오기

df_train_member = pd.read_parquet(r"C:\Users\rtc32\OneDrive\바탕 화면\final_project\project\1.회원정보_전처리완.parquet")

In [5]:
df_train_member.columns

Index(['기준년월', 'ID', '남녀구분코드', '연령', 'Segment', '회원여부_이용가능', '회원여부_이용가능_CA',
       '회원여부_이용가능_카드론', '소지여부_신용', '소지카드수_유효_신용', '소지카드수_이용가능_신용',
       '입회경과개월수_신용', '회원여부_연체', '이용거절여부_카드론', '동의여부_한도증액안내', '수신거부여부_TM',
       '수신거부여부_DM', '수신거부여부_메일', '수신거부여부_SMS', '가입통신회사코드', '탈회횟수_누적',
       '최종탈회후경과월', '탈회횟수_발급6개월이내', '탈회횟수_발급1년이내', '거주시도명', '직장시도명', '마케팅동의여부',
       '유효카드수_신용체크', '유효카드수_신용', '유효카드수_신용_가족', '유효카드수_체크', '유효카드수_체크_가족',
       '이용가능카드수_신용체크', '이용가능카드수_신용', '이용가능카드수_신용_가족', '이용가능카드수_체크',
       '이용가능카드수_체크_가족', '이용카드수_신용체크', '이용카드수_신용', '이용카드수_신용_가족', '이용카드수_체크',
       '이용금액_R3M_신용체크', '이용금액_R3M_신용', '이용금액_R3M_신용_가족', '이용금액_R3M_체크',
       '_1순위카드이용금액', '_1순위카드이용건수', '_1순위신용체크구분', '_2순위카드이용금액', '_2순위카드이용건수',
       '_2순위신용체크구분', '최종유효년월_신용_이용가능', '최종유효년월_신용_이용', '보유여부_해외겸용_본인',
       '이용가능여부_해외겸용_본인', '이용여부_3M_해외겸용_본인', '보유여부_해외겸용_신용_본인',
       '이용가능여부_해외겸용_신용_본인', '이용여부_3M_해외겸용_신용_본인', '연회비발생카드수_B0M', '기본연회비_B0M',
       '제휴연회비_B0M', '청구금액_기본연회비_B0M', '청구금액_제휴연회비_B

In [6]:
df_train_member = df_train_member[['ID', '기준년월', 'Segment']]


In [7]:
df_merged = pd.merge(df_trian, df_train_member, on=['ID', '기준년월'], how='left')

In [8]:
df_trian.shape

(2400000, 374)

In [9]:
df_merged.shape

(2400000, 375)

In [11]:
df_merged['Segment'].unique()

array(['D', 'E', 'C', 'A', 'B'], dtype=object)

In [12]:
from sklearn.preprocessing import LabelEncoder
from scipy.stats import spearmanr, chi2_contingency

def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    return np.sqrt(phi2 / min(k-1, r-1))

segment_col = 'Segment'
amount_col = '이용금액대'  # 순서형 변수

# Segment와 이용금액대 모두 순서형으로 라벨 인코딩
le_segment = LabelEncoder()
df_merged[segment_col] = le_segment.fit_transform(df_merged[segment_col])

le_amount = LabelEncoder()
df_merged[amount_col] = le_amount.fit_transform(df_merged[amount_col])

results = {}

for col in df_merged.columns:
    if col == segment_col:
        continue
    # 이용금액대(순서형)와 Segment(순서형) 조합이면 spearman
    if col == amount_col:
        coef, _ = spearmanr(df_merged[col], df_merged[segment_col], nan_policy='omit')
        results[col] = coef
    # datetime64 타입이면 int로 변환 후 spearman
    elif np.issubdtype(df_merged[col].dtype, np.datetime64):
        temp_col = df_merged[col].view('int64')
        coef, _ = spearmanr(temp_col, df_merged[segment_col], nan_policy='omit')
        results[col] = coef
    # 수치형이면 spearman
    elif pd.api.types.is_numeric_dtype(df_merged[col]):
        coef, _ = spearmanr(df_merged[col], df_merged[segment_col], nan_policy='omit')
        results[col] = coef
    # 범주형이면 cramer's v
    else:
        results[col] = cramers_v(df_merged[col], df_merged[segment_col])

results_series = pd.Series(results)
print(results_series)




C:\Users\rtc32\AppData\Local\Temp\ipykernel_2544\1498371094.py:33: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  temp_col = df_merged[col].view('int64')
C:\Users\rtc32\AppData\Local\Temp\ipykernel_2544\1498371094.py:33: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  temp_col = df_merged[col].view('int64')
C:\Users\rtc32\AppData\Local\Temp\ipykernel_2544\1498371094.py:33: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  temp_col = df_merged[col].view('int64')
C:\Users\rtc32\AppData\Local\Temp\ipykernel_2544\1498371094.py:33: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  temp_col = df_merged[col].view('int64')
C:\Users\rtc32\AppDa

기준년월               0.000000
ID                 1.000000
최종이용일자_기본         -0.089809
최종이용일자_신판         -0.084928
최종이용일자_CA          0.194113
                     ...   
승인거절건수_한도초과_R3M   -0.043431
승인거절건수_BL_R3M     -0.050384
승인거절건수_입력오류_R3M   -0.004663
승인거절건수_기타_R3M     -0.061179
이용금액대              0.481086
Length: 374, dtype: float64


In [15]:
# 소수점 4자리까지 고정 소수점 표기로 출력
pd.set_option('display.max_rows', None)  # 모든 행 출력
pd.set_option('display.float_format', '{:.4f}'.format)  # 지수 표기 대신 소수점 표기
print(results_series.reindex(results_series.abs().sort_values(ascending=False).index))

ID                       1.0000
정상청구원금_B5M              -0.5317
정상청구원금_B2M              -0.4920
정상청구원금_B0M              -0.4908
이용금액_일시불_R12M           -0.4900
이용금액대                    0.4811
최대이용금액_일시불_R12M         -0.4625
이용금액_오프라인_B0M           -0.4602
이용건수_신용_R12M            -0.4515
이용금액_일시불_B0M            -0.4513
이용금액_오프라인_R3M           -0.4509
이용금액_오프라인_R6M           -0.4499
이용금액_일시불_R6M            -0.4469
이용건수_신판_R12M            -0.4454
이용금액_일시불_R3M            -0.4440
이용건수_일시불_R12M           -0.4395
_2순위업종_이용금액             -0.4242
_3순위업종_이용금액             -0.4241
정상입금원금_B5M              -0.4139
_1순위업종_이용금액             -0.4121
_2순위쇼핑업종_이용금액           -0.4091
정상입금원금_B0M              -0.4073
_1순위쇼핑업종_이용금액           -0.4013
정상입금원금_B2M              -0.4001
이용건수_신용_R6M             -0.3993
이용건수_오프라인_R6M           -0.3977
이용건수_오프라인_B0M           -0.3945
이용건수_신용_R3M             -0.3931
이용건수_신용_B0M             -0.3930
_3순위쇼핑업종_이용금액           -0.3926
이용건수_오프라인_R3M           -0.3926
이용가맹점수  

In [21]:
filtered_df.columns

Index(['ID', '기준년월', '이용금액_일시불_B0M', '이용건수_신용_R12M', '이용건수_신판_R12M',
       '이용건수_일시불_R12M', '이용금액_일시불_R12M', '최대이용금액_일시불_R12M', '이용금액_일시불_R6M',
       '이용금액_일시불_R3M', '_1순위업종_이용금액', '_2순위업종_이용금액', '_3순위업종_이용금액',
       '_1순위쇼핑업종_이용금액', '_2순위쇼핑업종_이용금액', '이용금액_오프라인_R6M', '이용금액_오프라인_R3M',
       '이용금액_오프라인_B0M', '정상청구원금_B0M', '정상입금원금_B0M', '정상청구원금_B2M', '정상입금원금_B2M',
       '정상청구원금_B5M', '정상입금원금_B5M', '이용금액대'],
      dtype='object')

In [62]:
results_series[results_series.abs() >= 0.4]

ID                 1.0000
정상입금원금_B2M        -0.4001
_1순위쇼핑업종_이용금액     -0.4013
정상입금원금_B0M        -0.4073
_2순위쇼핑업종_이용금액     -0.4091
_1순위업종_이용금액       -0.4121
정상입금원금_B5M        -0.4139
_3순위업종_이용금액       -0.4241
_2순위업종_이용금액       -0.4242
이용건수_일시불_R12M     -0.4395
이용금액_일시불_R3M      -0.4440
이용건수_신판_R12M      -0.4454
이용금액_일시불_R6M      -0.4469
이용금액_오프라인_R6M     -0.4499
이용금액_오프라인_R3M     -0.4509
이용금액_일시불_B0M      -0.4513
이용건수_신용_R12M      -0.4515
이용금액_오프라인_B0M     -0.4602
최대이용금액_일시불_R12M   -0.4625
이용금액_일시불_R12M     -0.4900
정상청구원금_B0M        -0.4908
정상청구원금_B2M        -0.4920
정상청구원금_B5M        -0.5317
dtype: float64

In [23]:
# 1. 고정 순서로 포함할 컬럼
fixed_order = ['기준년월', 'ID']

# 2. 상관계수 절댓값 기준으로 필터링된 컬럼들
selected_cols = results_series[results_series.abs() >= 0.4].index.tolist()

# 3. 고정 컬럼 제외한 나머지 관련 컬럼
dynamic_cols = [col for col in selected_cols if col not in fixed_order]

# 4. 최종 순서
final_order = fixed_order + dynamic_cols

# 5. 순서대로 정렬된 DataFrame
filtered_df = df_merged[final_order]

In [55]:
filtered_df.columns

Index(['기준년월', 'ID', '이용금액_일시불_B0M', '이용건수_신용_R12M', '이용건수_신판_R12M',
       '이용건수_일시불_R12M', '이용금액_일시불_R12M', '최대이용금액_일시불_R12M', '이용금액_일시불_R6M',
       '이용금액_일시불_R3M', '_1순위업종_이용금액', '_2순위업종_이용금액', '_3순위업종_이용금액',
       '_1순위쇼핑업종_이용금액', '_2순위쇼핑업종_이용금액', '이용금액_오프라인_R6M', '이용금액_오프라인_R3M',
       '이용금액_오프라인_B0M', '정상청구원금_B0M', '정상입금원금_B0M', '정상청구원금_B2M', '정상입금원금_B2M',
       '정상청구원금_B5M', '정상입금원금_B5M', '이용금액대'],
      dtype='object')

In [59]:
unique_columns = list(set(a).symmetric_difference(set(filtered_df.columns)))
print(unique_columns)

[0.9999999999999997, -0.40014366525015843, -0.40729247915129196, -0.4091105595184029, -0.41207949544542644, -0.413871778724897, -0.42406366962433073, -0.42416002522771534, -0.4394712758814354, -0.45154444492830526, '이용금액_오프라인_R6M', '정상청구원금_B5M', '_3순위업종_이용금액', '_1순위업종_이용금액', '이용금액_일시불_R6M', -0.44993587758299725, -0.489987008657098, '이용금액_일시불_R3M', '_2순위쇼핑업종_이용금액', '정상청구원금_B0M', '_1순위쇼핑업종_이용금액', -0.5317261233141651, '정상입금원금_B5M', -0.4908253011839162, -0.4013161245750258, -0.4624778737205089, -0.44689155518301465, -0.44402249664094867, -0.46016279755427675, -0.4512878952903537, -0.45087409957574426, -0.44537337992301956, -0.4919860185941492, '이용금액_오프라인_B0M', '이용금액_일시불_B0M', '_2순위업종_이용금액', '이용금액_오프라인_R3M', '이용건수_신판_R12M', '이용건수_일시불_R12M', '이용건수_신용_R12M', '최대이용금액_일시불_R12M', '이용금액대', 'ID', '정상입금원금_B2M', '정상청구원금_B2M', '기준년월', '정상입금원금_B0M', '이용금액_일시불_R12M']


In [60]:
num_cols = len(filtered_df.columns)
print("컬럼 개수:", num_cols)

컬럼 개수: 25


In [61]:
filtered_df.columns

Index(['기준년월', 'ID', '이용금액_일시불_B0M', '이용건수_신용_R12M', '이용건수_신판_R12M',
       '이용건수_일시불_R12M', '이용금액_일시불_R12M', '최대이용금액_일시불_R12M', '이용금액_일시불_R6M',
       '이용금액_일시불_R3M', '_1순위업종_이용금액', '_2순위업종_이용금액', '_3순위업종_이용금액',
       '_1순위쇼핑업종_이용금액', '_2순위쇼핑업종_이용금액', '이용금액_오프라인_R6M', '이용금액_오프라인_R3M',
       '이용금액_오프라인_B0M', '정상청구원금_B0M', '정상입금원금_B0M', '정상청구원금_B2M', '정상입금원금_B2M',
       '정상청구원금_B5M', '정상입금원금_B5M', '이용금액대'],
      dtype='object')

In [27]:
#test 데이터도 동일한 컬럼으로 전처리
#결측치 확인
na_count = filtered_df.isna().sum()
for col, cnt in na_count.items():
    if cnt > 0:
        print(f"{col}: {cnt}개 결측치")

In [29]:
df77 = pd.read_parquet(r"C:\Users\rtc32\OneDrive\바탕 화면\final_project\train\3.승인매출정보\201807_train_승인매출정보.parquet")
df88= pd.read_parquet(r"C:\Users\rtc32\OneDrive\바탕 화면\final_project\train\3.승인매출정보\201808_train_승인매출정보.parquet")
df99= pd.read_parquet(r"C:\Users\rtc32\OneDrive\바탕 화면\final_project\train\3.승인매출정보\201809_train_승인매출정보.parquet")
df1010 = pd.read_parquet(r"C:\Users\rtc32\OneDrive\바탕 화면\final_project\train\3.승인매출정보\201810_train_승인매출정보.parquet")
df1111 = pd.read_parquet(r"C:\Users\rtc32\OneDrive\바탕 화면\final_project\train\3.승인매출정보\201811_train_승인매출정보.parquet")
df1212 = pd.read_parquet(r"C:\Users\rtc32\OneDrive\바탕 화면\final_project\train\3.승인매출정보\201812_train_승인매출정보.parquet")


In [30]:
dfs = [df77, df88, df99, df1010, df1111, df1212]
merged_train_df = pd.concat(dfs, ignore_index=True)

In [31]:
df_merged = pd.merge(merged_train_df, df_train_member, on=['ID', '기준년월'], how='left')

In [32]:
#메모리 사용 최소화를 위한 데이터 타입 변환 단, 수치형데이터 값을 확인했을때 단위가 일정이상 넘으면 사용하면 안됌
for col, dtype in df_merged.dtypes.items():
    if dtype == 'float64':
        df_merged[col] = df_merged[col].astype('float32')
    elif dtype == 'int64':
        df_merged[col] = df_merged[col].astype('int32')

In [33]:
#test 데이터도 동일한 컬럼으로 전처리
#결측치 확인
na_count = df_merged.isna().sum()
for col, cnt in na_count.items():
    if cnt > 0:
        print(f"{col}: {cnt}개 결측치")

_1순위업종: 539992개 결측치
_2순위업종: 912725개 결측치
_3순위업종: 1107898개 결측치
_1순위쇼핑업종: 922663개 결측치
_2순위쇼핑업종: 1135042개 결측치
_3순위쇼핑업종: 1312267개 결측치
_1순위교통업종: 1164494개 결측치
_2순위교통업종: 1656423개 결측치
_3순위교통업종: 2045455개 결측치
_1순위여유업종: 1987260개 결측치
_2순위여유업종: 2302286개 결측치
_3순위여유업종: 2377725개 결측치
_1순위납부업종: 1216263개 결측치
_2순위납부업종: 2033640개 결측치
_3순위납부업종: 2310187개 결측치
최종카드론_금융상환방식코드: 1958126개 결측치
최종카드론_신청경로코드: 1958226개 결측치
최종카드론_대출일자: 1988330개 결측치


In [50]:
cols_with_na = [col for col in df_merged.columns if df_merged[col].isna().sum() > 0]

for col in cols_with_na:
    print(f"== {col} 결측 아닌 행의 segment별 개수 ==")
    counts = df_merged[df_merged[col].notna()]['Segment'].value_counts()
    for seg, cnt in counts.items():
        print(f"{seg}: {cnt}개")
    print()  # 컬럼별 구분용 빈 줄

== _1순위업종 결측 아닌 행의 segment별 개수 ==
E: 1410572개
D: 327056개
C: 121286개
A: 950개
B: 144개

== _2순위업종 결측 아닌 행의 segment별 개수 ==
E: 1064402개
D: 306308개
C: 115488개
A: 939개
B: 138개

== _3순위업종 결측 아닌 행의 segment별 개수 ==
E: 891918개
D: 288912개
C: 110218개
A: 917개
B: 137개

== _1순위쇼핑업종 결측 아닌 행의 segment별 개수 ==
E: 1059086개
D: 302874개
C: 114306개
A: 927개
B: 144개

== _2순위쇼핑업종 결측 아닌 행의 segment별 개수 ==
E: 876108개
D: 280714개
C: 107091개
A: 909개
B: 136개

== _3순위쇼핑업종 결측 아닌 행의 segment별 개수 ==
E: 726866개
D: 259373개
C: 100477개
A: 883개
B: 134개

== _1순위교통업종 결측 아닌 행의 segment별 개수 ==
E: 851755개
D: 276000개
C: 106736개
A: 887개
B: 128개

== _2순위교통업종 결측 아닌 행의 segment별 개수 ==
E: 455807개
D: 202214개
C: 84656개
A: 782개
B: 118개

== _3순위교통업종 결측 아닌 행의 segment별 개수 ==
E: 186065개
D: 113190개
C: 54610개
A: 605개
B: 75개

== _1순위여유업종 결측 아닌 행의 segment별 개수 ==
E: 237190개
D: 117817개
C: 56943개
A: 684개
B: 106개

== _2순위여유업종 결측 아닌 행의 segment별 개수 ==
E: 43444개
D: 32523개
C: 21267개
A: 404개
B: 76개

== _3순위여유업종 결측 아닌 행의 segment별 개수 ==
D: 7619개
E: 7509개
C: 6894개
A:

In [63]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from scipy.stats import spearmanr, chi2_contingency

def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    return np.sqrt(phi2 / min(k - 1, r - 1))

# 순서형 변수 라벨 인코딩 (Segment와 이용금액대)
segment_col = 'Segment'
amount_col = '이용금액대'

# 원본 유지하고 복사
df = df_merged.copy()

# Segment 라벨 인코딩 (순서형)
le_segment = LabelEncoder()
df['Segment_encoded'] = le_segment.fit_transform(df[segment_col])

# 이용금액대 라벨 인코딩 (순서형)
le_amount = LabelEncoder()
df['이용금액대_encoded'] = le_amount.fit_transform(df[amount_col])

# 결과 저장
results = {}

# 결측치가 있는 컬럼만 대상으로 상관계수 계산
for col in df.columns:
    if col in [segment_col, 'Segment_encoded', '이용금액대', '이용금액대_encoded']:
        continue
    if df[col].isna().sum() == 0:
        continue  # 결측치 없는 컬럼은 건너뜀

    # 해당 컬럼 기준 결측치 제거
    temp_df = df[[col, 'Segment_encoded']].dropna()
    
    if temp_df.shape[0] < 2:
        results[col] = np.nan
        continue

    if np.issubdtype(df[col].dtype, np.datetime64):
        coef, _ = spearmanr(temp_df[col].view('int64'), temp_df['Segment_encoded'], nan_policy='omit')
        results[col] = coef
    elif pd.api.types.is_numeric_dtype(df[col]):
        coef, _ = spearmanr(temp_df[col], temp_df['Segment_encoded'], nan_policy='omit')
        results[col] = coef
    else:
        # 범주형 변수인 경우
        results[col] = cramers_v(temp_df[col], temp_df['Segment_encoded'])

# 결과 저장 및 출력
results_series = pd.Series(results).sort_values(ascending=False)
print(results_series)

_2순위여유업종          0.1317
_1순위업종            0.1161
_3순위여유업종          0.1052
_2순위업종            0.1024
_3순위쇼핑업종          0.0978
_1순위쇼핑업종          0.0973
_3순위업종            0.0917
_1순위여유업종          0.0850
_1순위교통업종          0.0843
최종카드론_금융상환방식코드    0.0842
_2순위쇼핑업종          0.0809
최종카드론_신청경로코드      0.0726
_2순위교통업종          0.0674
_1순위납부업종          0.0666
_3순위교통업종          0.0594
_3순위납부업종          0.0503
_2순위납부업종          0.0312
최종카드론_대출일자       -0.0424
dtype: float64


In [65]:
filtered_df.columns


Index(['기준년월', 'ID', '이용금액_일시불_B0M', '이용건수_신용_R12M', '이용건수_신판_R12M',
       '이용건수_일시불_R12M', '이용금액_일시불_R12M', '최대이용금액_일시불_R12M', '이용금액_일시불_R6M',
       '이용금액_일시불_R3M', '_1순위업종_이용금액', '_2순위업종_이용금액', '_3순위업종_이용금액',
       '_1순위쇼핑업종_이용금액', '_2순위쇼핑업종_이용금액', '이용금액_오프라인_R6M', '이용금액_오프라인_R3M',
       '이용금액_오프라인_B0M', '정상청구원금_B0M', '정상입금원금_B0M', '정상청구원금_B2M', '정상입금원금_B2M',
       '정상청구원금_B5M', '정상입금원금_B5M', '이용금액대'],
      dtype='object')

In [70]:
df_trian = df_trian[filtered_df.columns]
df_test = df_test[filtered_df.columns]

In [69]:
df_trian.dtypes

기준년월                int32
ID                 object
이용금액_일시불_B0M        int32
이용건수_신용_R12M        int32
이용건수_신판_R12M        int32
이용건수_일시불_R12M       int32
이용금액_일시불_R12M       int32
최대이용금액_일시불_R12M     int32
이용금액_일시불_R6M        int32
이용금액_일시불_R3M        int32
_1순위업종_이용금액         int32
_2순위업종_이용금액         int32
_3순위업종_이용금액         int32
_1순위쇼핑업종_이용금액       int32
_2순위쇼핑업종_이용금액       int32
이용금액_오프라인_R6M       int32
이용금액_오프라인_R3M       int32
이용금액_오프라인_B0M       int32
정상청구원금_B0M          int32
정상입금원금_B0M          int32
정상청구원금_B2M          int32
정상입금원금_B2M          int32
정상청구원금_B5M          int32
정상입금원금_B5M          int32
이용금액대              object
dtype: object

In [71]:
df_test.shape

(600000, 25)

In [75]:
df_test.dtypes

기준년월                int32
ID                 object
이용금액_일시불_B0M        int32
이용건수_신용_R12M        int32
이용건수_신판_R12M        int32
이용건수_일시불_R12M       int32
이용금액_일시불_R12M       int32
최대이용금액_일시불_R12M     int32
이용금액_일시불_R6M        int32
이용금액_일시불_R3M        int32
_1순위업종_이용금액         int32
_2순위업종_이용금액         int32
_3순위업종_이용금액         int32
_1순위쇼핑업종_이용금액       int32
_2순위쇼핑업종_이용금액       int32
이용금액_오프라인_R6M       int32
이용금액_오프라인_R3M       int32
이용금액_오프라인_B0M       int32
정상청구원금_B0M          int32
정상입금원금_B0M          int32
정상청구원금_B2M          int32
정상입금원금_B2M          int32
정상청구원금_B5M          int32
정상입금원금_B5M          int32
이용금액대              object
dtype: object

In [76]:
df_trian.to_parquet("3.승인매출정보_전처리완_2.parquet")
df_test.to_parquet("3.승인매출정보_test_전처리완_2.parquet")